# 데이터 불러오기

In [30]:
from torchtext.datasets import Multi30k
from torchtext.data import Field,BucketIterator # 데이터 전처리 내용을 명시함

import spacy
import torch
import numpy as np
import random

In [28]:
# 난수 생성
seed = 55

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128

In [7]:
spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

In [8]:
def tokenize_de(text): # 입력문장은 토큰화 후 순서를 뒤집는게 성능이 더 좋다고..
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text): # 타겟
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [17]:
src = Field(tokenize=tokenize_de,
           init_token = '<sos>',
           eos_token = '<eos>',
           lower = True)

trg = Field(tokenize=tokenize_en,
           init_token = '<sos>',
           eos_token = '<eos>',
           lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(src,trg))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:04<00:00, 246kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 82.8kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 76.4kB/s]


In [21]:
print(f'Number of training examples: {len(train_data.examples)}')
print(f'Number of validation examples: {len(valid_data.examples)}')
print(f'Number of testing examples: {len(test_data.examples)}')

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [23]:
src.build_vocab(train_data, min_freq=2) # 최소 2회 이상 등장한 토큰
trg.build_vocab(train_data, min_freq=2)

In [33]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=batch_size, device=device)

In [35]:
train_data.splits()

TypeError: splits() missing 2 required positional arguments: 'exts' and 'fields'

# 전처리

In [51]:
# 문자기준

In [45]:
sample.tar = sample.tar.apply(lambda x : '\t' + x + '\n') # SOS / EOS 기호

In [47]:
# 문자 집합
src_vocab = set()
for line in sample.src:
    for ch in line:
        src_vocab.add(ch)
        
tar_vocab = set()
for line in sample.tar:
    for ch in line:
        tar_vocab.add(ch)

In [48]:
src_vocab_size = len(src_vocab)+1 # +1 해주는 이유가 있었는데... 
tar_vocab_size = len(tar_vocab)+1

src_vocab = sorted(list(src_vocab)) # 인덱싱을 위한 작업
tar_vocab = sorted(list(tar_vocab))

In [49]:
# 인덱싱
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [58]:
# 단어기준
import unicodedata
import re

In [59]:
def to_ascii(s):
    # 프랑스어 악센트 삭제
    # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                      if unicodedata.category(c) !='Mn')

def preprocess_sentence(sent):
    # 악센트 제거 함수 호출
    sent = to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백 추가.
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [61]:
# test
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"


print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [93]:
# src 전처리
encoder_input = sample.src.apply(lambda x : preprocess_sentence(x).split()).tolist()

# tar 전처리
tar_line = sample.src.apply(preprocess_sentence).tolist()
# input 용 - 아마 teacher forcing 용?
decoder_input = list(map(lambda x : ("<sos> "+x).split(), tar_line))
decoder_target = list(map(lambda x : (x+" <eos>").split(), tar_line))

In [106]:
import spacy
from torchtext.legacy.data import Field

ModuleNotFoundError: No module named 'torchtext.legacy'

In [105]:
spacy_en = spacy.load('en_core_web_sm')
spacy_fr = spacy.load('fr_core_news_sm')

In [109]:
def tokenize_en(text) : 
    return [tok.text for tok in spacy_en.tokenizer(en_text)]

'A Dog Run back corner near spare bedrooms.'